In [11]:
import math

# HMM parameters
states = ['E', '5', 'I']
trans = {
    'E': {'E': math.log(0.9), '5': math.log(0.1), 'I': float('-inf')},
    '5': {'E': float('-inf'), '5': float('-inf'), 'I': math.log(1.0)},
    'I': {'E': float('-inf'), '5': float('-inf'), 'I': math.log(0.9)}
}
emit = {
    'E': {'A': math.log(0.25), 'C': math.log(0.25), 'G': math.log(0.25), 'T': math.log(0.25)},
    '5': {'A': math.log(0.05), 'C': float('-inf'), 'G': math.log(0.95), 'T': float('-inf')},
    'I': {'A': math.log(0.4), 'C': math.log(0.1), 'G': math.log(0.1), 'T': math.log(0.4)}
}

In [12]:
def viterbi(seq):
    n = len(seq)
    V = [{} for _ in range(n)]
    ptr = [{} for _ in range(n)]
    
    # Initialize first column
    for s in states:
        V[0][s] = emit[s][seq[0]] if s == 'E' else float('-inf')
    
    # Fill V and ptr tables
    for t in range(1, n):
        for s in states:
            max_p, best_prev = float('-inf'), ''
            for prev in states:
                p = V[t-1][prev] + trans[prev][s] + emit[s][seq[t]]
                if p > max_p:
                    max_p, best_prev = p, prev
            V[t][s], ptr[t][s] = max_p, best_prev
    
    # Traceback
    max_p, end_s = max((V[n-1][s], s) for s in states)
    path = [end_s]
    for t in range(n-1, 0, -1):
        end_s = ptr[t][end_s]
        path.insert(0, end_s)
    
    return ''.join(path)

In [13]:
def log_prob(seq, path):
    assert len(seq) == len(path), "Sequence and path lengths must match"
    p = emit[path[0]][seq[0]] if path[0] == 'E' else float('-inf')
    for t in range(1, len(seq)):
        p += trans[path[t-1]][path[t]] + emit[path[t]][seq[t]]
    p += math.log(0.1)  # Terminal transition
    return p

# Test
dna = "AGGCTCTCCATAAGG"
print(viterbi(dna))

test_seq = "CTTCATGTGAAAGCAGACGTAAGTCA"
test_path = "EEEEEEEEEEEEEEEEEE5IIIIIII"
print(f"Log probability: {log_prob(test_seq, test_path):.2f}")

EEEEEEEEEEEEEEE
Log probability: -41.22
